# Price and Markdown Optimization for Multiple Products

This model optimizes prices or markdown for multiple related products such as substitutable products within one category.

### Data
We use a simulator. No external datasets are used.

### Papers
1. Ferreira K., Simchi-Levi D., and Wang H. -- Online Network Revenue Management Using Thompson Sampling, November 2017

In [1]:
%matplotlib inline
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from scipy.optimize import linprog

def tabprint(msg, A):
    print(msg)
    print(tabulate(A, tablefmt="fancy_grid"))

# Markdown Optimization as LP Problem

We solve the following optimization problem:

$\text{max} \quad \sum_{i=1}^{N} \sum_{k=1}^{K} z_{ik} \cdot p_{k} \cdot q\left(i, p_{k}\right)$ 

$\text{subject to}$

$\quad \sum_{i=1}^{N} \sum_{k=1}^{K} z_{ik} \cdot q\left(i, p_{k}\right) = c$

$\quad \sum_{k=1}^{N} z_{ik} = 1, \quad \text{for}\ i=1,\ldots,N $

$\quad z_{ik}\ge 0 $

where q is demand, p is price, N is the number products, K is the number of price levels, and c is the sum of prices (the measure of the average price).

In [9]:
# prices - k-dimensional vector of allowed price levels 
# demands - n x k matrix of demand predictions
# c - required sum of prices 
# where k is the number of price levels, n is number of products
def optimal_prices_category(prices, demands, c):
    
    n, k = np.shape(demands)  

    # prepare inputs   
    r = np.multiply(np.tile(prices, n), np.array(demands).reshape(1, k*n))
    A = np.array([[
        1 if j >= k*(i) and j < k*(i+1) else 0
        for j in range(k*n)
    ] for i in range(n)])
    A = np.append(A, np.tile(prices, n), axis=0)
    b = [np.append(np.ones(n), c)]

    # solve the linear program
    res = linprog(-r.flatten(), 
              A_eq = A, 
              b_eq = b,  
              bounds=(0, 1))

    return np.array(res.x).reshape(n, k)

In [10]:
#
# Test run
#
r = optimal_prices_category(
  np.array([[1.00, 1.50, 2.00, 2.50]]),   # allowed price levels
  np.array([
    [ 28, 23, 20, 13],                    # demands for product 1
    [ 30, 22, 16, 12],                    # demands for product 2
    [ 32, 26, 19, 15]                     # demands for product 3
  ]), 5.50)                               # sum of prices for all three products

In [13]:
tabprint('prices = ', r)

prices = 
╒═════════════╤═════════════╤═════════════╤═════════════╕
│ 3.53009e-11 │ 2.68242e-10 │ 1           │ 3.29004e-10 │
├─────────────┼─────────────┼─────────────┼─────────────┤
│ 6.92591e-11 │ 1           │ 3.21738e-10 │ 2.62897e-10 │
├─────────────┼─────────────┼─────────────┼─────────────┤
│ 2.4468e-10  │ 0.5         │ 1.29104e-09 │ 0.5         │
╘═════════════╧═════════════╧═════════════╧═════════════╛


In [15]:
# output
# [[0  0     1   0  ]                            # price vector for product 1 ($2.00)
#  [0  1     0   0  ]                            # price vector for product 2 ($1.50)
#  [0  0.5   0   0.5]]                           # price vector for product 3 ($1.50 and 2.50)